# Subsetting and split-apply-combine

Subsetting: only load columns and rows that you need.

Split-apply-combine strategy:
- **split** your data in smaller subsets
- **apply** necessary transformation to subsets one at a time, storing transformation results
- **combine** results from subsets to get final result

In [ ]:
import os

import pandas as pd
import joblib

os.getpid()

### bigger than memory

Loading the whole example dataset in a DataFrame (20M rows, 15 cols) will take about 1 minute and occupy 4GB+ in memory. More memory will be used if you start running computations. Not feasible in Binder environment where memory is limited to 1-2GB, it will crash and restart your kernel. Multiply by 10 for actual InfoGroup.

In [ ]:
# running this cell in Binder will restart your kernel
df = []
for year in range(2001, 2021):
    df.append(pd.read_csv(f'data/synig/{year}.csv'))
df = pd.concat(df)

### use a subset for development and testing

If data rows are in random order, reading just the first few rows will give you a representative sample.

In [ ]:
df = []
for year in range(2001, 2021):
    df.append(pd.read_csv(f'data/synig/{year}.csv', nrows=10_000))
df = pd.concat(df)

In [ ]:
df.sample(10)

In [ ]:
df.groupby('SECTOR')['EMPLOYEES'].agg(['size', 'sum', 'mean']).astype(int).T

We are not getting all sectors of the economy here. Clearly, row order is not random.

### create a random sample

Let's create a random 1% sample. I will only use subset of years to save time.

In [ ]:
df = []
for year in range(2001, 2006):
    d = pd.read_csv(f'data/synig/{year}.csv')
    d = d.sample(frac=0.01)
    df.append(d)
df = pd.concat(df)

In [ ]:
df.shape

In [ ]:
df.sample(10)

Problem with this simple approach on our dataset: longitudinal histories are broken. It won't help if we could even load all years of data and sample from that. Solution: draw random sample of unique identifiers and then get full histories for those identifiers. This approach will yield a sample that has the same distribution as the original.

In [ ]:
abi = []
for year in range(2001, 2021):
    abi.append(pd.read_csv(f'data/synig/{year}.csv', usecols=['ABI']))
abi = pd.concat(abi)
abi = abi.drop_duplicates()
abi = abi.sample(frac=0.01)

The below cell will take about 1.5 minutes.

In [ ]:
df = []
for year in range(2001, 2021):
    d = pd.read_csv(f'data/synig/{year}.csv')
    d = d.merge(abi, 'left', 'ABI', indicator=True)
    d = d[d['_merge'] == 'both']
    del d['_merge']
    df.append(d)
    print('finished year', year)
df = pd.concat(df)

In [ ]:
df.sample(5)

We can use this lightweight sample to get some insights about the whole, for example, compare sector sizes.

In [ ]:
df.groupby('SECTOR')['EMPLOYEES'].agg(['size', 'sum', 'mean']).astype(int).T

### persist intermediate data for later use

You can save dataframe as CSV, `parquet` (stay tuned) or some other storage format. Or use standard Python `pickle` module. Here I am using `joblib`.

In [ ]:
joblib.dump(df, 'data/rand_1pct.pkl')
df.shape

Restart kernel and import modules.

In [ ]:
df1p = joblib.load('data/rand_1pct.pkl')
df1p.shape

## example 1: annual aggregates

For each year, compute total number of establishments, total and average employment.

### method 1: load all years into single dataframe

If we could fit data in memory, we would simply do this:

In [ ]:
df1p.groupby('YEAR')['EMPLOYEES'].agg(['size', 'sum', 'mean'])

Actually even with 1GB memory limit, we are able to load all years of data, because we only need two columns. Let's do this for  comparison with other methods.

*If this hits memory limit, try after restarting kernel or reduce number of years.*

In [ ]:
df = []
for year in range(2001, 2021):
    print(year, end=' ')
    d = pd.read_csv(f'data/synig/{year}.csv', usecols=['YEAR', 'EMPLOYEES'])
    df.append(d)
print()
df = pd.concat(df)
result = df.groupby('YEAR')['EMPLOYEES'].agg(['size', 'sum', 'mean'])

In [ ]:
result.astype(int).T

### method 2: split by year

Now let's use split-apply-combine. We load data year by year, compute aggregates and save them, and then combine.

In [ ]:
result = pd.DataFrame(index=['size', 'sum', 'mean'])
# split by year
for year in range(2001, 2021):
    print(year, end=' ')
    df = pd.read_csv(f'data/synig/{year}.csv', usecols=['EMPLOYEES'])
    # apply transformation
    res = df.agg(['size', 'sum', 'mean'])
    # combine year into final result
    result[year] = res
print()

In [ ]:
result.astype(int)

### method 3: split by year and chunking

Let's pretend that loading even a single year at a time is not feasible. We can use `pd.read_csv(chunksize=)` to only load a small chunk of the whole CSV. This becomes a nested split-apply-combine.

In [ ]:
result = pd.DataFrame(index=['size', 'sum', 'mean'])
# split by year
for year in range(2001, 2021):
    print(year, end=': ')
    res = pd.DataFrame(index=['size', 'sum'])
    # split by chunk
    for i, d in enumerate(pd.read_csv(f'data/synig/{year}.csv', usecols=['EMPLOYEES'], chunksize=100_000)):
        print(i, end=' ')
        # apply transformation
        # note: we don't compute mean here, because combining chunk means is not trivial
        res[i] = d.agg(['size', 'sum'])
        count_notna += d['EMPLOYEES'].notna().sum()
    # combine chunk results
    res = res.sum(1) 
    res['mean'] = res['sum'] / count_notna
    # combine year into final result
    result[year] = res
    print()

In [ ]:
result.astype(int)

### measuring

We will use `ResourceMonitor` to compare resource usage between three methods.

To make sure that no data is hanging in memory between runs, I wrap each method in a function, so dataframes can be garbage collected once function returns.

In [ ]:
# restart kernel before running
import time
import pandas as pd
from tools import ResourceMonitor

years = range(2001, 2021)

def method1_load_all():
    print('Start method 1')
    t0 = time.time()
    df = []
    for year in years:
        print(year, end=' ')
        d = pd.read_csv(f'data/synig/{year}.csv', usecols=['YEAR', 'EMPLOYEES'])
        df.append(d)
    print()
    df = pd.concat(df)
    result = df.groupby('YEAR')['EMPLOYEES'].agg(['size', 'sum', 'mean'])
    dt = time.time() - t0
    print(f'Finish method 1 in {dt:.1f} seconds\n')
    return result.T

def method2_split_by_year():
    print('Start method 2')
    t0 = time.time()
    result = pd.DataFrame(index=['size', 'sum', 'mean'])
    for year in years:
        print(year, end=' ')
        df = pd.read_csv(f'data/synig/{year}.csv', usecols=['EMPLOYEES'])
        res = df.agg(['size', 'sum', 'mean'])
        result[year] = res
    print()
    dt = time.time() - t0
    print(f'Finish method 2 in {dt:.1f} seconds\n')
    return result

def method3_split_by_year_chunk():
    print('Start method 3')
    t0 = time.time()
    result = pd.DataFrame(index=['size', 'sum', 'mean'])
    for year in years:
        print(year, end=': ')
        res = pd.DataFrame(index=['size', 'sum'])
        count_notna = 0
        for i, d in enumerate(pd.read_csv(f'data/synig/{year}.csv', usecols=['EMPLOYEES'], chunksize=100_000)):
            print(i, end=' ')
            res[i] = d.agg(['size', 'sum'])
            count_notna += d['EMPLOYEES'].notna().sum()
        res = res.sum(1) 
        res['mean'] = res['sum'] / count_notna
        result[year] = res
        print()
    dt = time.time() - t0
    print(f'Finish method 3 in {dt:.1f} seconds\n')
    return result


mon = ResourceMonitor()
mon.start()
time.sleep(1) # give monitor time to start
mon.tag('method 1 V')
r1 = method1_load_all()
time.sleep(1)
mon.tag('method 2 V')
r2 = method2_split_by_year()
time.sleep(1)
mon.tag('method 3 V')
r3 = method3_split_by_year_chunk()
time.sleep(1)
mon.stop()

# make sure that all methods yield same results
assert (r1 == r2).all().all()
assert (r1 == r3).all().all()

In [ ]:
mon.plot()

We can take closer look at data stored by monitor.

|                 | Method 1 | Method 2 | Method 3 |
|-----------------|:--------:|:--------:|:--------:|
| Code complexity |    low   |  medium  |   high   |
| Running time    |   fast   |  medium  |   slow   |
| Memory usage    |   high   |  medium  |    low   |

## example 2: size vs age